# Create StationXML File

        ENTER INSTRUMENT DETAILS AND STATION DETAILS
        --------------------------------------------
        stnm   - station code e.g. ABCD
        ntwk   - network code e.g. XX
        chan   - channel code e.g. HHZ
        locode - location code e.g. 00 or leave blank 
        stla   - station latitude (degree decimal) e.g. -65.9876
        stlo   - station longitude (degree decimal) e.g. -130.7654
        stel   - station elevation above MSL (meters) e.g. 354
        stdp   - depth of the instrument from the surface (meters) e.g. 10
        cmpaz  - component azimuth: vertical = 0, north = 0, east = 90
        cmpinc - component incident angle: vertical up = -90, horizontal = 0 
        Fs     - sampling rate (i.e. 1/dt)  e.g. if dt = 0.025s, then Fs = 40
        stdate - start data + time of the instrument = ('yyyy,mm,dd,hh,mn,sec.decimal') e.g. ('2016,1,2,08,30,55.99')
        endate - end date + time of the instrument   = ('yyyy,mm,dd,hh,mn,sec.decimal') e.g. ('2017,1,2,23,30,55.99')
        sitenm - site name e.g. Gippsland
        stcomm - comments about the station e.g. At the top of a hill
        sensornm   - name of the sensor e.g. Streckeisen or Guralp
        sensortype - type of the sensor eg. STS-1
        sensorT    - natural period of the sensor e.g. 360 seconds
        senvty     - some sensors have sensitivity as a variable
        sensorkey  - number of strings required to uniquely identify a response for an instrument
                     if sensornm, sensortype, sensorT, and senvty are used for the key, sensorkey = 4
                     if sensornm, sensortype, and sensorT are used for the key, sensorkey = 3
                     if sensornm and sensortype are used for the key, sensorkey = 2
                     
        * data logger information can be included but ignored here because Kelunji Geckos and EchoPros are not 
          included in the NRL. Thus, include this manually as described later. This is a critical step. 
        
        * Use the TEST AREAS at the bottom of this script to find out station specific information from the mseed 
          files and/or NRL if these are unknown.
        
        OUTPUT: stnm_ntwk_chan.xml
   
   
        SENSOR KEYS
        -----------
        sensornm, sensortype, sensorT, and sometimes senvty strings consitute the unique sensor key to retrive sensor 
        responses from the IRIS DMC Library of Nominal Responses for Seismic Instruments (NRL). Thus, these need to 
        be declared exactly. sensorkey declares how many of these strings are used to define a unique sensor key.   
        
        If you are unclear about what to use for keys, follow these steps in the last cell of this script.
        
        e.g. using an instrument from the manufacturer Nanometrics, which is your sensornm
            
        STEP 1: Find the available instrument types 
        
                    from obspy.clients.nrl import NRL
                    sensornm = 'Nanometrics'
                    print(nrl.sensors[sensornm])
                    
                This will output the available instrument types. For example, Nanometrics has;
                
                    'Meridian Compact Posthole', 'Meridian Posthole',
                    'Titan Accelerometer', 'Trillium 120 Horizon', 'Trillium 120P/PA',
                    'Trillium 120Q/QA,PH,BH (Vault, Posthole or Borehole)',
                    'Trillium 240', 'Trillium 360', 'Trillium 40',
                    'Trillium All-Terrain', 'Trillium Cascadia Accelerometer',
                    'Trillium Cascadia seismometer', 'Trillium Compact 120 (Vault,
                     Posthole, OBS)', 'Trillium Compact 20 (Vault, Posthole, OBS)'
                     
        STEP 2: Pick your instrument from this list, which is your sensortype and repeat the print statement by 
                including it;
          
                     from obspy.clients.nrl import NRL
                     sensornm = 'Nanometrics'
                     sensortype = 'Meridian Posthole'
                     print(nrl.sensors[sensornm][sensortype])
                    
                Check if this outputs a list of periods (e.g. '20 seconds', '360 seconds'). If a list is present,
                pick the appropriate one for your instrument, which is your sensorT. In this case set selected 
                sensornm, sensortype, sensorT, and sensorkey = 3. On the other hand, if a list is not provided (in 
                this example 'Meridian Posthole, 120 s, 1200 V/m/s' ...., is printed), then sensornm and sensortype 
                are sufficient to set up a unique sensor key and sensorT is ignored. If this is the case, set 
                selected sensornm, sensortype, and sensorkey = 2. 
                
                
        STEP 3: If the instrument has a sensitivity as an additional variable, then available choices will show up as 
                a list when STEP 2 is complete. For example,
                                    
                                    sensornm   = 'Guralp'
                                    sensortype = 'CMG-6T'
                                    sensorT    = '10s - 100Hz'
                                    print(nrl.sensors[sensornm][sensortype][sensorT])
                                    
                                    produces the following list of sensitivities,
                
                                    Select the sensor sensitivity in V/m/s (4 items):
                                    '2000', '20000', '2400', '4000'
                
                Pick one from these sensitivity and declare senvty as well and set sensorkey = 4
                
                
       For the Nanmetrics example used here, you will set up variables as follows;
                
                    sensornm   = 'Nanometrics'
                    sensortype = 'Meridian Posthole'
                    sensorT    = 'ignored, you can type anything here'
                    senvty     = 'ignored, you can type anything here'
                    sensorkey  = 2
                     
 
       
       MODIFYING INSTRUMENT STAGE GAIN FOR DATA LOGGER
       -----------------------------------------------
       It is CRITICAL that the user knows counts per volts (cpv) conversion of the data logger exactly. For the data 
       loggers available in the NRL, this can be automatically included in the response (not done in this script). 
       For Kelunji Geckos and EchoPros cpv = 419430.4. This data logger conversion should reflect in <StageGain> 
       contained within the block <Stage number="1"> (i.e., stage 1) in the StationXML file.   
       
       The output of this script will contain only the instrument sensitivity in the above <StageGain>. For instance, 
       
       For a Guralp CMG-6T 1s - 100 Hz 2400 instrument, this would be as follows;
       
       <StageGain>
              <Value>2400</Value>
              <Frequency>5.0</Frequency>
       </StageGain>
       
       To include the data logger conversion, the instrument sensitivity (in the example 2400) must be multiplied by 
       419430.4 for Kelunji Geckos and EchoPros. Add this modification to the StationXML file manually. After this 
       modification, the StationXML file 
       will have
       
       (1) for a Guralp CMG-6T 1s - 100 Hz 2400 instrument (2400 x 419430.4)
           <StageGain>
                  <Value>1006632960</Value>
                  <Frequency>5.0</Frequency>
           </StageGain>
       
       (2) for a Guralp CMG-3ESP 60 s - 50 Hz 2000 instrument (2000 x 419430.4)
           <StageGain>
                  <Value>838860800</Value>
                  <Frequency>1.0</Frequency>
           </StageGain>
       
       (3) For the Nanometrics Compact 20 instrument (753.1x419430.4 = 315873034.24)
           <StageGain>
                  <Value>315873034.24</Value>
                  <Frequency>1.0</Frequency>
           </StageGain>
       
 
       UoM INSTRUMENT TYPES
       --------------------
       
       copy and paste the appropriate instrument types to create StationXML files. 
       
               (a) Guralp CMG-6T: 
                    sensornm   = 'Guralp'
                    sensortype = 'CMG-6T'
                    sensorT    = '1s - 100Hz'
                    senvty     = '2400'
                    sensorkey  = 4
                    
                    e.g. WPSH
                    
               (b) Guralp CMG-3ESP
                    sensornm   = 'Guralp'
                    sensortype = 'CMG-3ESP'
                    sensorT    = '60 s - 50 Hz'
                    senvty     = '2000'
                    sensorkey  = 4
                    
                    e.g. CLIF
       
               (c)  sensornm   = 'Nanometrics'
                    sensortype = 'Trillium Compact 20 (Vault, Posthole, OBS)'
                    sensorT    = '753 V/m/s'
                    senvty     = ''
                    sensorkey  = 3
                    
                    e.g. DDNE, DDSW, DDWK, LRNW





        Januka Attanayake
        University of Melbourne
        07 JUNE 2019

In [20]:
###############################################################
#                       USER INPUT                            #
###############################################################
stnm       = 'LRNW'
ntwk       = 'UM'
locode     = ''
creator    = 'NRL'
#-----------------
chan       = 'CHE'
cmpaz      = 90
cmpinc     = 0
#-----------------
stla       = -38.22918
stlo       = 147.274075
stel       = 26
stdp       = 0
Fs         = 250
stdate     = '2013,1,2,0,0,0.00'
endate     = '2599,1,2,0,0,0.00'
sitenm     = 'Gippsland'
stcomm     = 'Belongs to GipNet'
#-----------------
sensornm   = 'Nanometrics'
sensortype = 'Trillium Compact 20 (Vault, Posthole, OBS)'
sensorT    = '753 V/m/s'
senvty     = ''
sensorkey  = 3




###############################################################
#             START BUILDING THE STATIONXML FILE              #
###############################################################
import obspy
from obspy.core.inventory import Inventory, Network, Station, Channel, Site
from obspy.clients.nrl import NRL
from obspy import read



inv = Inventory(networks=[], source=creator)
net = Network(code=ntwk, stations=[], description=stcomm, start_date=obspy.UTCDateTime(stdate))
sta = Station(code=stnm, latitude=stla, longitude=stlo,elevation=stel, creation_date=obspy.UTCDateTime(stdate),
              site=Site(name=sitenm))
cha = Channel(code=chan, location_code=locode, latitude=stla, longitude=stlo, elevation=stel, depth=stdp,
              azimuth=cmpaz, dip=cmpinc, sample_rate=Fs)



###############################################################
#      QUERY NRL LIBRARY AND GET RESPONSE INFORMATION         #
###############################################################
nrl = NRL()

if sensorkey == 4:
    response = nrl.get_sensor_response(sensor_keys=[sensornm, sensortype, sensorT, senvty])
    
elif sensorkey == 3:
    response = nrl.get_sensor_response(sensor_keys=[sensornm, sensortype,sensorT])
    
else:
    response = nrl.get_sensor_response(sensor_keys=[sensornm, sensortype])



    
###############################################################
#    PUT ALL PIECES TOGETHER AND WRITE OUT THE STATIONXML     #
###############################################################   
cha.response = response
sta.channels.append(cha)
net.stations.append(sta)
inv.networks.append(net)

outfile = (stnm + '_' + ntwk + '_' + chan + '.xml')
inv.write(outfile, format="stationxml", validate=True)

# Test area 1 for selecting unknown intrument keys

In [6]:
from obspy.clients.nrl import NRL
nrl = NRL()

sensornm = 'Guralp'
sensortype = 'CMG-3esp'
sT = '1s - 100Hz'
svt = '2400'

sensornm = 'Nanometrics'
sensortype = 'Trillium Compact 20 (Vault, Posthole, OBS)'
sT = '753 V/m/s'

sensornm = 'Guralp'
sensortype = 'CMG-3ESP'
sT = '60 s - 50 Hz'
svt = '2000'

#print(nrl.sensors[sensornm][sensortype][sT][svt])
print(nrl.sensors[sensornm][sensortype][sT])

Select the sensor sensitivity in V/m/s (3 items):
  '1500', '2000', '20000'


# Test area 2 for retrieving stnm, ntwk, chan, Fs, and locode from the mseed file

In [ ]:
stnm       = 'DDNE'
chan       = 'CHZ'
mseed_data = 'data/20181225_24hr_Z_DDNE.mseed'
staxns     = read(mseed_data)
seis       = staxns.select(station=stnm)
print(seis)
#print('Number of traces:',len(staxns.__str__()))

# Plotting
seis.plot()
ax.xaxis_date()